⚡ Intermediaire | ⏱ 45 min | 🔑 Concepts : yield, generateurs, lazy evaluation

# Generateurs et Lazy Evaluation

## Objectifs

- Comprendre la difference entre `yield` et `return`
- Creer et utiliser des fonctions generatrices
- Maitriser le protocole d'iteration (`__iter__`, `__next__`)
- Optimiser l'utilisation de la memoire avec les generateurs
- Construire des pipelines de traitement de donnees
- Comprendre le lien avec Spark et la lazy evaluation

## Prerequis

- Fonctions en Python
- Boucles for et iterables
- List comprehensions et expressions generatrices

## 1. yield vs return

### Fonction normale avec return

Une fonction normale execute son code et **retourne une valeur** avec `return`. Elle se termine ensuite.

In [ ]:
def nombres_return():
    """Fonction normale avec return"""
    resultat = []
    for i in range(5):
        resultat.append(i)
    return resultat

# Appel de la fonction
valeurs = nombres_return()
print(f"Type: {type(valeurs)}")
print(f"Valeurs: {valeurs}")
print(f"Taille en memoire: {valeurs.__sizeof__()} bytes")

### Fonction generatrice avec yield

Une fonction contenant `yield` devient un **generateur**. Elle **suspend son execution** et peut etre **reprise** plus tard.

In [ ]:
def nombres_yield():
    """Fonction generatrice avec yield"""
    for i in range(5):
        print(f"  [Generateur] Avant yield {i}")
        yield i
        print(f"  [Generateur] Apres yield {i}")

# Appel de la fonction generatrice
gen = nombres_yield()
print(f"Type: {type(gen)}")
print(f"Contenu: {gen}")
print(f"Taille en memoire: {gen.__sizeof__()} bytes")

print("\nConsommation du generateur:")
for valeur in gen:
    print(f"Recu: {valeur}")

**Observations importantes:**

1. `nombres_yield()` retourne un objet generateur, pas une liste
2. Le code n'est **pas execute** immediatement
3. Chaque `yield` **suspend** la fonction et **retourne** une valeur
4. L'execution **reprend** apres le `yield` au prochain appel

## 2. Fonctions generatrices

Une fonction generatrice est une fonction qui contient au moins un `yield`.

In [ ]:
def compte_a_rebours(n):
    """Generateur de compte a rebours"""
    print(f"Debut du compte a rebours depuis {n}")
    while n > 0:
        yield n
        n -= 1
    print("Decollage!")

# Utilisation
for nombre in compte_a_rebours(5):
    print(f"  {nombre}...")

In [ ]:
def pairs_infinis():
    """Generateur infini de nombres pairs"""
    n = 0
    while True:  # Boucle infinie!
        yield n
        n += 2

# Utilisation avec limite
gen_pairs = pairs_infinis()
for i in range(10):
    print(next(gen_pairs), end=' ')
print()

In [ ]:
def fibonacci(max_termes):
    """Generateur de la suite de Fibonacci"""
    a, b = 0, 1
    compte = 0
    while compte < max_termes:
        yield a
        a, b = b, a + b
        compte += 1

# Utilisation
fib = list(fibonacci(15))
print(f"15 premiers termes de Fibonacci: {fib}")

## 3. Protocole d'iteration

### Les concepts cles

- **Iterable**: objet qu'on peut parcourir (liste, tuple, dict, set, generateur, etc.)
- **Iterator**: objet qui produit des valeurs une par une
- **`__iter__()`**: methode qui retourne un iterator
- **`__next__()`**: methode qui retourne la prochaine valeur

In [ ]:
# Une liste est un iterable
ma_liste = [1, 2, 3, 4, 5]

# Obtenir un iterator depuis l'iterable
mon_iterator = iter(ma_liste)
print(f"Type de l'iterator: {type(mon_iterator)}")

# Utiliser next() pour obtenir les valeurs une par une
print(f"Premier: {next(mon_iterator)}")
print(f"Deuxieme: {next(mon_iterator)}")
print(f"Troisieme: {next(mon_iterator)}")

### StopIteration

Quand un iterator est epuise, il leve l'exception `StopIteration`.

In [ ]:
petite_liste = [1, 2]
it = iter(petite_liste)

print(next(it))  # 1
print(next(it))  # 2

# Le prochain appel levera StopIteration
try:
    print(next(it))
except StopIteration:
    print("Iterator epuise!")

### Creer son propre iterator

In [ ]:
class CompteurPairs:
    """Iterator qui compte les nombres pairs"""
    
    def __init__(self, maximum):
        self.maximum = maximum
        self.nombre = 0
    
    def __iter__(self):
        """Retourne l'iterator (lui-meme)"""
        return self
    
    def __next__(self):
        """Retourne la prochaine valeur"""
        if self.nombre > self.maximum:
            raise StopIteration
        
        valeur = self.nombre
        self.nombre += 2
        return valeur

# Utilisation
for n in CompteurPairs(10):
    print(n, end=' ')
print()

# Equivalent avec un generateur (beaucoup plus simple!)
def compteur_pairs_gen(maximum):
    nombre = 0
    while nombre <= maximum:
        yield nombre
        nombre += 2

print("\nAvec generateur:")
for n in compteur_pairs_gen(10):
    print(n, end=' ')
print()

## 4. Expressions generatrices vs list comprehensions

Rappel: une **expression generatrice** utilise des parentheses `()` au lieu de crochets `[]`.

In [ ]:
import sys

# List comprehension - cree toute la liste
liste_carres = [x**2 for x in range(1000)]
print(f"Liste: {sys.getsizeof(liste_carres):,} bytes")

# Expression generatrice - cree juste le generateur
gen_carres = (x**2 for x in range(1000))
print(f"Generateur: {sys.getsizeof(gen_carres):,} bytes")

# Difference enorme pour grandes sequences
grande_liste = [x for x in range(1_000_000)]
grand_gen = (x for x in range(1_000_000))

print(f"\nGrande liste (1M elements): {sys.getsizeof(grande_liste):,} bytes")
print(f"Grand generateur (1M elements): {sys.getsizeof(grand_gen):,} bytes")
print(f"Ratio: {sys.getsizeof(grande_liste) / sys.getsizeof(grand_gen):.0f}x plus petit")

In [ ]:
# Utilisation pratique: traiter de grandes donnees
def traiter_fichier_liste(n):
    """Simule la lecture d'un fichier en liste (charge tout en memoire)"""
    return [f"Ligne {i}" for i in range(n)]

def traiter_fichier_gen(n):
    """Simule la lecture d'un fichier en generateur (une ligne a la fois)"""
    for i in range(n):
        yield f"Ligne {i}"

# Avec liste - tout en memoire
print("Avec liste (charge tout):")
lignes_liste = traiter_fichier_liste(5)
for ligne in lignes_liste:
    print(f"  {ligne}")

# Avec generateur - une ligne a la fois
print("\nAvec generateur (lazy):")
lignes_gen = traiter_fichier_gen(5)
for ligne in lignes_gen:
    print(f"  {ligne}")

## 5. Pipelines de generateurs

On peut **chainer** des generateurs pour creer des pipelines de traitement efficaces.

In [ ]:
def lire_nombres(n):
    """Generateur source: produit des nombres"""
    print("[lire_nombres] Demarrage")
    for i in range(n):
        print(f"  [lire_nombres] Yielding {i}")
        yield i

def filtrer_pairs(nombres):
    """Filtre: garde uniquement les pairs"""
    print("[filtrer_pairs] Demarrage")
    for n in nombres:
        if n % 2 == 0:
            print(f"  [filtrer_pairs] Yielding {n}")
            yield n

def doubler(nombres):
    """Transformation: double chaque nombre"""
    print("[doubler] Demarrage")
    for n in nombres:
        resultat = n * 2
        print(f"  [doubler] Yielding {resultat}")
        yield resultat

# Construire le pipeline (rien n'est execute encore!)
print("=== Construction du pipeline ===")
nombres = lire_nombres(6)
pairs = filtrer_pairs(nombres)
doubles = doubler(pairs)

print("\n=== Execution du pipeline ===")
# L'execution commence seulement ici
for valeur in doubles:
    print(f">>> Resultat final: {valeur}\n")

In [ ]:
# Exemple reel: traitement de logs
def lire_logs():
    """Simule la lecture d'un fichier de logs"""
    logs = [
        "INFO: Application demarree",
        "DEBUG: Connexion base de donnees OK",
        "ERROR: Fichier non trouve",
        "INFO: Traitement termine",
        "ERROR: Timeout connexion",
        "WARNING: Memoire faible",
    ]
    for log in logs:
        yield log

def filtrer_erreurs(logs):
    """Garde uniquement les erreurs"""
    for log in logs:
        if log.startswith('ERROR'):
            yield log

def extraire_message(logs):
    """Extrait le message apres le ':'"""
    for log in logs:
        _, message = log.split(': ', 1)
        yield message

# Pipeline de traitement
logs = lire_logs()
erreurs = filtrer_erreurs(logs)
messages = extraire_message(erreurs)

print("Erreurs trouvees:")
for msg in messages:
    print(f"  - {msg}")

## 6. send() et close() (Avance)

Les generateurs peuvent aussi **recevoir** des valeurs avec `send()` et etre **fermes** avec `close()`.

In [ ]:
def moyenne_mobile():
    """Calcule une moyenne mobile"""
    total = 0
    count = 0
    moyenne = None
    
    while True:
        valeur = yield moyenne
        if valeur is not None:
            total += valeur
            count += 1
            moyenne = total / count

# Utilisation
calc = moyenne_mobile()
next(calc)  # Demarrer le generateur

print(f"Moyenne apres 10: {calc.send(10)}")
print(f"Moyenne apres 20: {calc.send(20)}")
print(f"Moyenne apres 30: {calc.send(30)}")
print(f"Moyenne apres 40: {calc.send(40)}")

# Fermer le generateur
calc.close()

## 7. Lazy vs Eager Evaluation

### Eager Evaluation (evaluation immediate)

Le resultat est **calcule immediatement** et **stocke en memoire**.

In [ ]:
# Eager: list comprehension
print("=== EAGER (list comprehension) ===")
print("Creation de la liste...")
carres_eager = [x**2 for x in range(5)]
print(f"Liste creee: {carres_eager}")
print(f"Type: {type(carres_eager)}")
print("\nParcours de la liste:")
for c in carres_eager:
    print(f"  {c}")

### Lazy Evaluation (evaluation paresseuse)

Le resultat est **calcule a la demande**, seulement quand on en a besoin.

In [ ]:
# Lazy: generateur
print("=== LAZY (generateur) ===")
print("Creation du generateur...")
carres_lazy = (x**2 for x in range(5))
print(f"Generateur cree: {carres_lazy}")
print(f"Type: {type(carres_lazy)}")
print("\nParcours du generateur (calcul a la demande):")
for c in carres_lazy:
    print(f"  {c}")

### Avantages de la lazy evaluation

1. **Economie de memoire**: ne stocke pas toutes les valeurs
2. **Performance**: peut eviter des calculs inutiles
3. **Sequences infinies**: possibles avec lazy, impossibles avec eager

In [ ]:
# Exemple: chercher le premier element qui satisfait une condition
def nombres_lents(n):
    """Simule un calcul lent"""
    import time
    for i in range(n):
        time.sleep(0.1)  # Simule un calcul long
        print(f"  Calcul de {i}...")
        yield i

# Eager: calcule TOUT avant de chercher
print("EAGER: calcule tout d'abord")
liste = list(nombres_lents(10))  # Attend 1 seconde (10 * 0.1)
premier_grand = next(x for x in liste if x > 5)
print(f"Premier > 5: {premier_grand}")

# Lazy: s'arrete des qu'on trouve
print("\nLAZY: s'arrete des qu'on trouve")
gen = nombres_lents(10)
premier_grand = next(x for x in gen if x > 5)
print(f"Premier > 5: {premier_grand}")
# Seulement 7 calculs au lieu de 10!

## 8. Avantages memoire: traiter de gros fichiers

Les generateurs sont essentiels pour traiter des fichiers trop gros pour tenir en memoire.

In [ ]:
# MAUVAIS: charge tout le fichier en memoire
def lire_fichier_eager(fichier):
    with open(fichier, 'r') as f:
        lignes = f.readlines()  # Charge TOUT en memoire
    return lignes

# BON: lit ligne par ligne
def lire_fichier_lazy(fichier):
    with open(fichier, 'r') as f:
        for ligne in f:  # Une ligne a la fois
            yield ligne.strip()

# Creer un fichier de test
import tempfile
import os

with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.txt') as f:
    fichier_test = f.name
    for i in range(100):
        f.write(f"Ligne {i}\n")

# Utilisation lazy
print("Lecture lazy (generateur):")
count = 0
for ligne in lire_fichier_lazy(fichier_test):
    if count < 5:
        print(f"  {ligne}")
    count += 1
print(f"Total: {count} lignes")

# Nettoyer
os.unlink(fichier_test)

In [ ]:
# Pipeline de traitement de fichier CSV
def lire_csv(fichier):
    """Lit un fichier CSV ligne par ligne"""
    with open(fichier, 'r') as f:
        for ligne in f:
            yield ligne.strip()

def parser_csv(lignes):
    """Parse chaque ligne CSV"""
    for ligne in lignes:
        yield ligne.split(',')

def filtrer_colonnes(lignes, indices):
    """Garde uniquement certaines colonnes"""
    for ligne in lignes:
        yield [ligne[i] for i in indices]

# Creer un CSV de test
with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.csv') as f:
    csv_test = f.name
    f.write("nom,age,ville,pays\n")
    f.write("Alice,25,Paris,France\n")
    f.write("Bob,30,Londres,UK\n")
    f.write("Charlie,35,Berlin,Allemagne\n")

# Pipeline de traitement
lignes = lire_csv(csv_test)
donnees = parser_csv(lignes)
noms_villes = filtrer_colonnes(donnees, [0, 2])  # nom et ville seulement

print("Noms et villes:")
for nom, ville in noms_villes:
    print(f"  {nom} habite a {ville}")

os.unlink(csv_test)

## 9. Lien avec Spark: Lazy Evaluation

**Apache Spark** utilise le meme principe de lazy evaluation que les generateurs Python.

### Similitudes:

1. **Transformations lazy**: construisent un plan d'execution sans executer
2. **Actions eager**: declenchent l'execution
3. **Optimisation**: le plan peut etre optimise avant execution
4. **Economie memoire**: ne charge pas tout en memoire

In [ ]:
# Exemple conceptuel (sans Spark reel)
class FakeSpark:
    """Simule le comportement lazy de Spark"""
    
    def __init__(self, data):
        self.data = data
        self.operations = []
    
    def map(self, func):
        """Transformation LAZY"""
        print(f"  [LAZY] Ajout transformation map")
        new_spark = FakeSpark(self.data)
        new_spark.operations = self.operations + [('map', func)]
        return new_spark
    
    def filter(self, func):
        """Transformation LAZY"""
        print(f"  [LAZY] Ajout transformation filter")
        new_spark = FakeSpark(self.data)
        new_spark.operations = self.operations + [('filter', func)]
        return new_spark
    
    def collect(self):
        """Action EAGER - execute tout"""
        print("  [EAGER] Execution de toutes les operations...")
        result = self.data
        for op_name, func in self.operations:
            if op_name == 'map':
                result = [func(x) for x in result]
            elif op_name == 'filter':
                result = [x for x in result if func(x)]
        return result

# Utilisation
print("Construction du pipeline (lazy):")
rdd = FakeSpark([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
rdd2 = rdd.filter(lambda x: x % 2 == 0)  # LAZY
rdd3 = rdd2.map(lambda x: x ** 2)  # LAZY

print("\nExecution (eager):")
resultat = rdd3.collect()  # EAGER - execute maintenant
print(f"Resultat: {resultat}")

## 10. Module itertools

Le module `itertools` fournit des outils pour creer des iterateurs efficaces.

In [ ]:
import itertools

# count: compteur infini
print("count(10, 2) - commence a 10, increment de 2:")
compteur = itertools.count(10, 2)
for i in range(5):
    print(f"  {next(compteur)}")

# cycle: repete une sequence indefiniment
print("\ncycle(['A', 'B', 'C']) - repete indefiniment:")
cycle = itertools.cycle(['A', 'B', 'C'])
for i in range(8):
    print(f"  {next(cycle)}")

# repeat: repete une valeur
print("\nrepeat('Hello', 3) - repete 3 fois:")
for val in itertools.repeat('Hello', 3):
    print(f"  {val}")

In [ ]:
# chain: enchaine plusieurs iterables
print("chain([1,2,3], [4,5,6]) - enchaine les sequences:")
chaine = itertools.chain([1, 2, 3], [4, 5, 6], [7, 8, 9])
print(f"  {list(chaine)}")

# islice: coupe un iterable
print("\nislice(range(100), 5, 15, 2) - elements 5 a 15, pas de 2:")
tranche = itertools.islice(range(100), 5, 15, 2)
print(f"  {list(tranche)}")

# takewhile: prend tant que la condition est vraie
print("\ntakewhile(lambda x: x<5, [1,2,3,4,5,6,1,2]) - prend tant que <5:")
pris = itertools.takewhile(lambda x: x < 5, [1, 2, 3, 4, 5, 6, 1, 2])
print(f"  {list(pris)}")

# dropwhile: ignore tant que la condition est vraie
print("\ndropwhile(lambda x: x<5, [1,2,3,4,5,6,7,3,2]) - ignore tant que <5:")
ignore = itertools.dropwhile(lambda x: x < 5, [1, 2, 3, 4, 5, 6, 7, 3, 2])
print(f"  {list(ignore)}")

In [ ]:
# compress: filtre selon un selecteur
print("compress('ABCDEF', [1,0,1,0,1,1]) - garde ou selecteur=1:")
compresse = itertools.compress('ABCDEF', [1, 0, 1, 0, 1, 1])
print(f"  {''.join(compresse)}")

# accumulate: cumule les valeurs
print("\naccumulate([1,2,3,4,5]) - somme cumulee:")
cumul = itertools.accumulate([1, 2, 3, 4, 5])
print(f"  {list(cumul)}")

# accumulate avec operation personnalisee
print("\naccumulate([1,2,3,4,5], operator.mul) - produit cumule:")
import operator
produit = itertools.accumulate([1, 2, 3, 4, 5], operator.mul)
print(f"  {list(produit)}")

## Pieges courants

### 1. Generateur epuise (ne peut etre reitere)

In [ ]:
# PIEGE: un generateur ne peut etre parcoru qu'une fois
gen = (x**2 for x in range(5))

print("Premier parcours:")
print(list(gen))

print("\nDeuxieme parcours:")
print(list(gen))  # Vide! Le generateur est epuise

# SOLUTION: recreer le generateur ou convertir en liste
gen = (x**2 for x in range(5))
liste = list(gen)  # Convertir une fois

print("\nAvec liste:")
print(liste)
print(liste)  # Fonctionne!

### 2. Debugging difficile

In [ ]:
# PIEGE: impossible de voir le contenu d'un generateur directement
gen = (x**2 for x in range(5))
print(f"Generateur: {gen}")  # Pas tres utile!

# SOLUTION 1: convertir en liste pour debug
gen = (x**2 for x in range(5))
print(f"\nContenu (liste): {list(gen)}")

# SOLUTION 2: utiliser islice pour voir les premiers elements
gen = (x**2 for x in range(100))
premiers = list(itertools.islice(gen, 5))
print(f"\n5 premiers elements: {premiers}")

### 3. Variables capturees dans les generateurs

In [ ]:
# PIEGE: les variables sont capturees par reference
def creer_gen_piege():
    gens = []
    for i in range(3):
        gens.append((x * i for x in range(3)))  # i est capture par reference!
    return gens

gens = creer_gen_piege()
print("Avec piege:")
for i, g in enumerate(gens):
    print(f"  Gen {i}: {list(g)}")  # Tous utilisent i=2!

# SOLUTION: capturer la valeur immediatement
def creer_gen_ok():
    gens = []
    for i in range(3):
        # Capturer i dans une fonction lambda avec argument par defaut
        gens.append((x * val for x in range(3)) for val in [i])
        # OU creer une fonction generatrice
    return gens

# Meilleure solution: fonction generatrice
def creer_gen_meilleur():
    def gen_pour_i(i):
        return (x * i for x in range(3))
    
    return [gen_pour_i(i) for i in range(3)]

gens = creer_gen_meilleur()
print("\nCorrect:")
for i, g in enumerate(gens):
    print(f"  Gen {i}: {list(g)}")

## Mini-exercices

### Exercice 1: Generateur Fibonacci

Creez un generateur qui produit la suite de Fibonacci jusqu'a une valeur maximale (pas un nombre de termes).

In [ ]:
def fibonacci_max(max_valeur):
    """Genere la suite de Fibonacci jusqu'a max_valeur"""
    # Votre code ici
    pass

# Test
print("Fibonacci jusqu'a 100:")
for n in fibonacci_max(100):
    print(n, end=' ')
print()

### Exercice 2: Lecture lazy de fichier

Creez un pipeline de generateurs pour:
1. Lire un fichier ligne par ligne
2. Filtrer les lignes vides
3. Convertir en majuscules
4. Compter les mots

In [ ]:
def lire_lignes(fichier):
    """Lit un fichier ligne par ligne"""
    # Votre code ici
    pass

def filtrer_vides(lignes):
    """Filtre les lignes vides"""
    # Votre code ici
    pass

def en_majuscules(lignes):
    """Convertit en majuscules"""
    # Votre code ici
    pass

def compter_mots(lignes):
    """Compte les mots de chaque ligne"""
    # Votre code ici
    pass

# Test (creez d'abord un fichier de test)

### Exercice 3: Pipeline de transformation

Creez un pipeline qui:
1. Genere les nombres de 1 a 100
2. Garde uniquement les multiples de 3 ou 5
3. Calcule le carre de chaque nombre
4. Ne garde que les nombres < 1000
5. Retourne la somme

In [ ]:
# Votre code ici


## Solutions

### Solution Exercice 1

In [ ]:
def fibonacci_max(max_valeur):
    """Genere la suite de Fibonacci jusqu'a max_valeur"""
    a, b = 0, 1
    while a <= max_valeur:
        yield a
        a, b = b, a + b

# Test
print("Fibonacci jusqu'a 100:")
for n in fibonacci_max(100):
    print(n, end=' ')
print()

print("\nFibonacci jusqu'a 1000:")
print(list(fibonacci_max(1000)))

# Bonus: somme des Fibonacci pairs jusqu'a 4 millions (Projet Euler #2)
somme = sum(n for n in fibonacci_max(4_000_000) if n % 2 == 0)
print(f"\nSomme des Fibonacci pairs jusqu'a 4M: {somme:,}")

### Solution Exercice 2

In [ ]:
def lire_lignes(fichier):
    """Lit un fichier ligne par ligne"""
    with open(fichier, 'r', encoding='utf-8') as f:
        for ligne in f:
            yield ligne.rstrip('\n')

def filtrer_vides(lignes):
    """Filtre les lignes vides"""
    for ligne in lignes:
        if ligne.strip():  # Ignore lignes vides ou avec seulement espaces
            yield ligne

def en_majuscules(lignes):
    """Convertit en majuscules"""
    for ligne in lignes:
        yield ligne.upper()

def compter_mots(lignes):
    """Compte les mots de chaque ligne"""
    for ligne in lignes:
        nb_mots = len(ligne.split())
        yield (ligne, nb_mots)

# Creer un fichier de test
with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.txt', encoding='utf-8') as f:
    fichier_test = f.name
    f.write("Bonjour le monde\n")
    f.write("\n")
    f.write("Python est genial\n")
    f.write("   \n")
    f.write("Les generateurs sont puissants\n")

# Pipeline complet
lignes = lire_lignes(fichier_test)
non_vides = filtrer_vides(lignes)
majuscules = en_majuscules(non_vides)
avec_comptage = compter_mots(majuscules)

print("Resultat du pipeline:")
for ligne, nb in avec_comptage:
    print(f"  [{nb} mots] {ligne}")

os.unlink(fichier_test)

### Solution Exercice 3

In [ ]:
# Solution avec generateurs explicites
def nombres():
    """Genere les nombres de 1 a 100"""
    for i in range(1, 101):
        yield i

def multiples_3_ou_5(nombres):
    """Garde uniquement les multiples de 3 ou 5"""
    for n in nombres:
        if n % 3 == 0 or n % 5 == 0:
            yield n

def au_carre(nombres):
    """Calcule le carre de chaque nombre"""
    for n in nombres:
        yield n ** 2

def inferieurs_1000(nombres):
    """Garde uniquement les nombres < 1000"""
    for n in nombres:
        if n < 1000:
            yield n

# Pipeline
pipeline = inferieurs_1000(au_carre(multiples_3_ou_5(nombres())))
resultat = sum(pipeline)
print(f"Somme (avec pipeline): {resultat}")

# Solution compacte avec expressions generatrices
resultat_compact = sum(
    n**2 
    for n in range(1, 101) 
    if (n % 3 == 0 or n % 5 == 0) and n**2 < 1000
)
print(f"Somme (compact): {resultat_compact}")

# Verification: afficher les valeurs
valeurs = [
    n**2 
    for n in range(1, 101) 
    if (n % 3 == 0 or n % 5 == 0) and n**2 < 1000
]
print(f"\nValeurs: {valeurs}")
print(f"Nombre de valeurs: {len(valeurs)}")

## Ressources supplementaires

- [PEP 255 - Simple Generators](https://www.python.org/dev/peps/pep-0255/)
- [PEP 342 - Coroutines via Enhanced Generators](https://www.python.org/dev/peps/pep-0342/)
- Documentation Python: [itertools](https://docs.python.org/3/library/itertools.html)
- Documentation Python: [Functional Programming HOWTO](https://docs.python.org/3/howto/functional.html)